In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("best_one").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [6]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

X, y = make_classification(n_samples=1000, random_state=0)
#X[:5]

param_grid = {"C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
             "kernel": ['rbf', 'poly', 'sigmoid'],
             "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                          param_grid=param_grid,
                          return_train_score=False,
                          cv=3)

In [7]:
grid_search.fit(X, y)

GridSearchCV(cv=3,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                         'kernel': ['rbf', 'poly', 'sigmoid'],
                         'shrinking': [True, False]})

In [25]:
# запуск кода SKLEARN в PYSPARK
from sklearn.utils import parallel_backend
from joblibspark import register_spark

register_spark() # регистрация Spark

with parallel_backend('spark',n_jobs=3):
    grid_search.fit(X, y)

In [27]:
# на Spark
from sklearn.utils import parallel_backend
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
from joblibspark import register_spark

register_spark() #

iris = datasets.load_iris()
clf = svm.SVC(kernel='linear', C=1)
with parallel_backend('spark', n_jobs=3):
    scores = cross_val_score(clf, iris.data, iris.target, cv=5)

print(scores)

[0.96666667 1.         0.96666667 0.96666667 1.        ]


### нельзя отправить на Spark модели(

In [ ]:
from sklearn.feature_extraction import FeatureHasher

h = FeatureHasher(n_features=10)

with parallel_backend('spark', n_jobs=3):
    # Не будет в Spark - будет на локальной машине ((
    h.transform()

from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

with parallel_backend('spark', n_jobs=3):
    # Не будет в Spark - будет на локальной машине ((
    regr.predict(diabetes_X_test)